# Amazon SageMaker Processing を用いた Recruit Restaurant Visitor Forecasting 解法の機械学習パイプライン化

このノートブックでは、AWS のマネージドな機械学習基盤サービスである、Amazon SageMaker を活用して、前処理や学習、推論を行います。 特にSageMaker の機能である SageMaker Processing を活用すると、 一般的な Docker コンテナによる機械学習の実行方法から大きく変更することなく、マネージドな機械学習基盤を活用できるので便利です。 今回は Kaggle で行われたレストランの訪問者数を予測する [Recruit Restaurant Visitor Forecasting](https://www.kaggle.com/c/recruit-restaurant-visitor-forecasting) コンペを題材に、その解法を AWS を用いて実行するようなパイプラインにする手段を見ていきましょう。今回は Deiscussion にある [8th place solution write-up](https://www.kaggle.com/c/recruit-restaurant-visitor-forecasting/discussion/49166) を参考にしました。解法は solution.ipynb にまとめられていますが、機械学習パイプラインとしての取り回しを良くするために、前処理、ハイパーパラメータの調整、学習と推論のそれぞれのステップに分割しました。詳細は 
`./scripts` 以下をご確認ください。

In [ ]:
import pandas as pd

import sagemaker
from sagemaker import get_execution_role
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

# SageMaker を活用するための権限が付与された Role を準備します。
role = get_execution_role()

## 処理を実行するための Docker イメージを準備

In [ ]:
!cat ./scripts/Dockerfile

In [ ]:
!docker build -t kaggle-reqruit-sagemaker ./scripts

In [ ]:
import boto3

# boto3の機能を使ってリポジトリ名に必要な情報を取得する
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
tag = ':latest'

# Repository 名の中に sagemaker が含まれている必要がある
ecr_repository = f'kaggle-reqruit-sagemaker'
image_uri = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{ecr_repository+tag}'

!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
 
# リポジトリの作成
# すでにある場合はこのコマンドは必要ない
!aws ecr create-repository --repository-name $ecr_repository
 
!docker build -t {ecr_repository} .
!docker tag {ecr_repository + tag} $image_uri
!docker push $image_uri

print(f'コンテナは {image_uri} へ登録されています。')

## データの前処理

In [ ]:
sagemaker_session = sagemaker.Session()
s3_bucket_path = 's3://' + sagemaker_session.default_bucket()

job_name = f'reqruit-pipeline-preprocess'
preprocessing_input_s3 = s3_bucket_path + '/reqruit-pipeline/data'
preprocessing_output_s3 = s3_bucket_path + '/reqruit-pipeline/preprocessed_data'

processing_input_dir = '/opt/ml/processing/input'
processing_output_dir = '/opt/ml/processing/output'

In [ ]:
processor = ScriptProcessor(
                    base_job_name=job_name,
                    image_uri=image_uri,
                    command=['python3'],
                    role=role,
                    instance_count=1,
                    instance_type='ml.c5.xlarge'
                          )

In [ ]:
processor.run(
    code='./scripts/preprocess/preprocess.py', # S3 の URI でも可
    inputs=[ProcessingInput(source=preprocessing_input_s3, destination=processing_input_dir)],
    outputs=[ProcessingOutput(source=processing_output_dir, destination=preprocessing_output_s3)],
    arguments=[
          '--input_dir',processing_input_dir,
          '--output_dir',processing_output_dir
              ]
            )

## ハイパーパラメータのチューニング
解法ではハイパーパラメータは調整済みで、既に固定されていましたが、ビジネス上でこのようなモデルを運用する場合には、都度ハイパラのチューニングを行うことがあるのではないでしょうか。今回は [Optuna](https://github.com/optuna/optuna) を使用して調整することにしました。

In [ ]:
job_name = f'reqruit-pipeline-hpo'
hpo_input_s3 = s3_bucket_path + '/reqruit-pipeline/preprocessed_data'
hpo_output_s3 = s3_bucket_path + '/reqruit-pipeline/hpo'

hpo_input_dir = '/opt/ml/processing/input'
hpo_output_dir = '/opt/ml/processing/output'

In [ ]:
processor = ScriptProcessor(
                    base_job_name=job_name,
                    image_uri=image_uri,
                    command=['python3'],
                    role=role,
                    instance_count=1,
                    instance_type='ml.c5.2xlarge'
                          )

In [ ]:
processor.run(
    code='./scripts/hpo/hpo.py', # S3 の URI でも可
    inputs=[ProcessingInput(source=hpo_input_s3, destination=hpo_input_dir)],
    outputs=[ProcessingOutput(source=hpo_output_dir, destination=hpo_output_s3)],
    arguments=[
          '--input_dir', hpo_input_dir,
          '--output_dir', hpo_output_dir,
          '--n_trials', "1",
              ]
            )

## 学習と推論

In [ ]:
job_name = f'reqruit-pipeline-train-predict'
train_predict_input_s3 = s3_bucket_path + '/reqruit-pipeline/hpo'
train_predict_output_s3 = s3_bucket_path + '/reqruit-pipeline/train_predict'

train_predict_input_dir = '/opt/ml/processing/input'
train_predict_output_dir = '/opt/ml/processing/output'

In [ ]:
processor = ScriptProcessor(
                    base_job_name=job_name,
                    image_uri=image_uri,
                    command=['python3'],
                    role=role,
                    instance_count=1,
                    instance_type='ml.c5.2xlarge'
                          )

In [ ]:
processor.run(
    code='./scripts/train_predict/train_predict.py', # S3 の URI でも可
    inputs=[ProcessingInput(source=train_predict_input_s3, destination=train_predict_input_dir)],
    outputs=[ProcessingOutput(source=train_predict_output_dir, destination=train_predict_output_s3)],
    arguments=[
          '--input_dir', processing_input_dir,
          '--output_dir', processing_output_dir
              ]
            )